In [2]:
import os
import random
import xml.etree.ElementTree as ET
import math
import pprint
import pdb
import json
import numpy as np
import sys
from keras_frcnn import config
sys.path.insert(1, '/home/mh/opencv-master/build/lib/python3')
import cv2

sys.setrecursionlimit(40000)
C = config.Config()
C.use_horizontal_flips = False
C.use_vertical_flips = False


def format_img(img):
	img_min_side = 600.0
	(height,width,_) = img.shape
	
	if width <= height:
		f = img_min_side/width
		new_height = int(f * height)
		new_width = int(img_min_side)
	else:
		f = img_min_side/height
		new_width = int(f * width)
		new_height = int(img_min_side)
	img = cv2.resize(img,(new_width,new_height), interpolation=cv2.INTER_CUBIC)
	img = img[:,:,(2,1,0)]
	img = np.transpose(img,(2,0,1)).astype(np.float32)
	img = np.expand_dims(img, axis=0)
	img[:, 0, :, :] -= 103.939
	img[:, 1, :, :] -= 116.779
	img[:, 2, :, :] -= 123.68
	return img

with open('classes.json', 'r') as class_data_json:
    class_mapping = json.load(class_data_json)

if 'bg' not in class_mapping:
	class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}

class_to_color = {class_mapping[v]:np.random.randint(0,255,3) for v in class_mapping}
num_rois = 16

import keras_frcnn.resnet as nn
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras_frcnn import roi_helpers

if K.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
	input_shape_features = (1024, None, None)
else:
	input_shape_img = (None, None, 3)
	input_shape_features = (None, None, 1024)


img_input = Input(shape=input_shape_img)

feature_map_input = Input(shape=input_shape_features)

roi_input = Input(shape=(num_rois, 4))

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn = nn.rpn(shared_layers,num_anchors)

# classifier, uses base layers + proposals
print(class_mapping)

classifier = nn.classifier(feature_map_input,roi_input,num_rois,nb_classes=len(class_mapping))

model_rpn = Model(img_input,rpn + [shared_layers])
model_classifier = Model([feature_map_input,roi_input],classifier)

# weights_path = 'model_frcnn.hdf5'
weights_path = '/home/mh/ws/fish_challenge/input/models/frcnn_1_7_3epochs.h5'

model_rpn.load_weights(weights_path, by_name=True)
model_classifier.load_weights(weights_path, by_name=True)

model_rpn.compile(optimizer='sgd',loss='mse')
model_classifier.compile(optimizer='sgd',loss='mse')

all_imgs = []

classes = {}

visualise = True

Using TensorFlow backend.
/home/mh/ws/fish_challenge/Fishy-master/notbook/keras_frcnn/resnet.py:123: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, shortcut], mode='sum')
/home/mh/.local/lib/python3.5/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/mh/ws/fish_challenge/Fishy-master/notbook/keras_frcnn/resnet.py:50: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, input_tensor], mode='sum')


{0: 'alb', 1: 'bet', 2: 'dol', 3: 'lag', 4: 'other', 5: 'shark', 6: 'yft', 7: 'bg'}


/home/mh/ws/fish_challenge/Fishy-master/notbook/keras_frcnn/resnet.py:161: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, shortcut], mode='sum')
/home/mh/ws/fish_challenge/Fishy-master/notbook/keras_frcnn/resnet.py:84: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, input_tensor], mode='sum')


In [3]:
%matplotlib inline

img_path = '/home/mh/ws/fish_challenge/input/train/ALB'
LAG = '/home/mh/ws/fish_challenge/input/train/LAG'
bufsize = 0

img_name = LAG+'/img_01952.jpg'

img = cv2.imread(img_name)

X = format_img(img)

img_scaled = np.transpose(X[0,(2,1,0),:,:],(1,2,0)).copy()
img_scaled[:, :, 0] += 123.68
img_scaled[:, :, 1] += 116.779
img_scaled[:, :, 2] += 103.939

img_scaled = img_scaled.astype(np.uint8)

if K.image_dim_ordering() == 'tf':
    X = np.transpose(X,(0,2,3,1))
# get the feature maps and output from the RPN


[Y1, Y2, F] = model_rpn.predict(X)
print('RPN output, F')
# print(Y1)
# print(Y2)
print(F[0])
R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_dim_ordering())
print('roi shape: ')
print(R.shape)

# convert from (x1,y1,x2,y2) to (x,y,w,h)
R[:,2] = R[:,2] - R[:,0]
R[:,3] = R[:,3] - R[:,1]

# apply the spatial pyramid pooling to the proposed regions
bboxes = {}
probs = {}
for jk in range(R.shape[0]//num_rois + 1):
    ROIs = np.expand_dims(R[num_rois*jk:num_rois*(jk+1),:],axis=0)
    print('ROIs shape after expanding', ROIs.shape)
    if ROIs.shape[1] == 0:
        break

    if jk == R.shape[0]//num_rois:
        #pad R
        curr_shape = ROIs.shape
        target_shape = (curr_shape[0],num_rois,curr_shape[2])
        ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
        ROIs_padded[:,:curr_shape[1],:] = ROIs
        ROIs_padded[0,curr_shape[1]:,:] = ROIs[0,0,:]
        ROIs = ROIs_padded

        
    [P_cls, P_regr] = model_classifier.predict([F, ROIs])
    # print(P_cls)
    # print('Look up!!!!!!!!!')
    P_regr = P_regr / C.std_scaling

    for ii in range(P_cls.shape[1]):

        if np.max(P_cls[0,ii,:]) < 0.5 or np.argmax(P_cls[0,ii,:]) == (P_cls.shape[2] - 1):
            continue

        cls_name = class_mapping[np.argmax(P_cls[0,ii,:])]
        print(cls_name)
        if cls_name not in bboxes:
            bboxes[cls_name] = []
            probs[cls_name] = []

        (x, y, w, h) = ROIs[0,ii,:]

        cls_num = np.argmax(P_cls[0, ii, :])
        (tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
        x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)

        bboxes[cls_name].append([16*x, 16*y, 16*(x+w), 16*(y+h)])
        probs[cls_name].append(np.max(P_cls[0, ii, :]))

all_dets = {}
print(np.array(bboxes).shape)
for key in bboxes:
    bbox = np.array(bboxes[key])
    print(bbox)
    new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlapThresh=0.2)
    for jk in range(new_boxes.shape[0]):
        (x1,y1,x2,y2) = new_boxes[jk,:]
        print('BOunding box ', x1,x2,y1,y2)

        cv2.rectangle(img_scaled,(x1, y1), (x2, y2), class_to_color[key],2)

        textLabel = '{}:{}'.format(key,int(100*new_probs[jk]))
        if key not in all_dets:
            all_dets[key] = 100*new_probs[jk]
        else:
            all_dets[key] = max(all_dets[key],100*new_probs[jk])

        (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
        textOrg = (x1,y1+20)

        cv2.rectangle(img_scaled,(textOrg[0] - 5,textOrg[1]+baseLine - 5),(textOrg[0]+retval[0] + 5,textOrg[1]-retval[1] - 5),(0,0,0),2)
        cv2.rectangle(img_scaled,(textOrg[0] - 5,textOrg[1]+baseLine - 5),(textOrg[0]+retval[0] + 5,textOrg[1]-retval[1] - 5),(255,255,255),-1)
        cv2.putText(img_scaled,textLabel,textOrg,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1)
cv2.imshow('img',img_scaled)
cv2.waitKey(0)

RPN output, F
[[[  0.          28.23772049  12.44805622 ...,   3.03905821   3.01053095
    18.25119019]
  [  8.47764397  25.70064354  10.565135   ...,   5.7929821    2.21841717
    21.41402054]
  [  2.38297319  23.14270973  10.90889263 ...,  12.8384552    4.39076757
    20.7801342 ]
  ..., 
  [  0.           8.21832466   4.1822567  ...,   1.96847737   1.72690046
     9.41632843]
  [  2.34821773  16.97750092   4.90823603 ...,   4.6421032    1.90400195
    12.81979275]
  [  6.19673681  18.99401093   5.65789986 ...,   4.5287962    1.47181356
     9.34779167]]

 [[  0.          11.99928856  11.383955   ...,   9.10273361   4.70348167
     5.10096645]
  [  7.03823566   6.25314951  12.10570431 ...,   6.41989803   1.37008834
    10.80335426]
  [  7.88896608   5.18575764  11.41356182 ...,   5.72576284   2.51321268
    14.27163029]
  ..., 
  [  0.32901597   2.56770611   4.20784998 ...,   2.6576252    1.53747797
     5.09799433]
  [  0.9285605    0.75087237   5.86465406 ...,   3.2249012    3.3486

86